In [1]:
import pandas as pd
import numpy as np
import requests
import json

In this jupyter notebook we read the daily and hourly foot traffic data, convert it do Pandas dataframes,  
handle data gaps, and add columns for school holidays, black week, etc.

In [2]:
# read daily foot traffic data, convert to Pandas dataframe, fill data gaps with NaN, add columns for day of week and month
dft = pd.read_csv('DATA/foottraffic_daily_schoenbornstr.csv',sep=';',parse_dates=['Zeitstempel'])
df2 = dft[['Zeitstempel', 'Passanten','Passanten ltr','Passanten rtl']]
df2 = df2.sort_values(by='Zeitstempel')
sdate = '2020-01-01'
edate = '2024-12-31'
df2 = df2[(df2['Zeitstempel'] >= sdate) & (df2['Zeitstempel'] <= edate)]
df2['date_diff'] = df2['Zeitstempel'].diff()
for idx, row in df2.iterrows():
    if pd.Timedelta(row['date_diff']) != pd.Timedelta('1 days'):
        display(row)
# data gaps are never larger than 1 day --> fill data gaps with NaN
for idx, row in df2.iterrows():
    if pd.Timedelta(row['date_diff']) == pd.Timedelta('2 days'):
        new_row = {
            'Zeitstempel': row['Zeitstempel'] - pd.Timedelta(days=1),
            'Passanten': np.nan,
            'Passanten ltr': np.nan,
            'Passanten rtl': np.nan,
            'date_diff': pd.NaT
        }
        df2 = pd.concat([df2.iloc[:idx+1], pd.DataFrame([new_row]), df2.iloc[idx+1:]]).reset_index(drop=True)
df2 = df2.sort_values('Zeitstempel').reset_index(drop=True)
df = df2[['Zeitstempel', 'Passanten','Passanten ltr','Passanten rtl']]
df = df.rename(columns={'Zeitstempel': 'date'})
df = df.rename(columns={'Passanten': 'ftraffic'})
df = df.rename(columns={'Passanten ltr': 'ltr'})
df = df.rename(columns={'Passanten rtl': 'rtl'})
df['dow'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month_name()
display(df.head())

Zeitstempel      2020-01-01 00:00:00
Passanten                       7285
Passanten ltr                    NaN
Passanten rtl                    NaN
date_diff                        NaT
Name: 29, dtype: object

Zeitstempel      2024-03-07 00:00:00
Passanten                      23513
Passanten ltr                    NaN
Passanten rtl                    NaN
date_diff            2 days 00:00:00
Name: 1529, dtype: object

Zeitstempel      2024-07-30 00:00:00
Passanten                      28590
Passanten ltr                13948.0
Passanten rtl                14642.0
date_diff            2 days 00:00:00
Name: 1653, dtype: object

Zeitstempel      2024-09-22 00:00:00
Passanten                       8827
Passanten ltr                 4123.0
Passanten rtl                 4704.0
date_diff            2 days 00:00:00
Name: 1702, dtype: object

,date,ftraffic,ltr,rtl,dow,month
0,2020-01-01,7285.0,NaN,NaN,Wednesday,January
1,2020-01-02,40406.0,NaN,NaN,Thursday,January
2,2020-01-03,41095.0,NaN,NaN,Friday,January
3,2020-01-04,42096.0,NaN,NaN,Saturday,January
4,2020-01-05,7847.0,NaN,NaN,Sunday,January


In [3]:
# read hourly foot traffic data, convert to Pandas dataframe, fill data gaps with NaN, add columns for day of week and month
dft = pd.read_csv('DATA/foottraffic_hourly_schoenbornstr.csv',sep=';',parse_dates=['Zeitstempel'])
df2 = dft[['Zeitstempel', 'Passanten','Passanten ltr','Passanten rtl']]
df2 = df2.sort_values(by='Zeitstempel')
df2['Zeitstr'] = df2['Zeitstempel'].astype(str).str[:19]
df2['Zeitstr'] = pd.to_datetime(df2['Zeitstr'])
df2 = df2.drop(columns=['Zeitstempel'])
df2 = df2[['Zeitstr'] + [col for col in df2.columns if col != 'Zeitstr']]
sdate = '2024-01-01 00:00:00'
edate = '2024-12-31 23:00:00'
df2 = df2[(df2['Zeitstr'] >= sdate) & (df2['Zeitstr'] <= edate)]
df2['date_diff'] = df2['Zeitstr'].diff()
rdf = []
rdf.append([df2.iloc[0]['Zeitstr'], df2.iloc[0]['Passanten'], df2.iloc[0]['Passanten ltr'], df2.iloc[0]['Passanten rtl']])
for idx in range(1, len(df2)):
    row1 = df2.iloc[idx-1]
    row2 = df2.iloc[idx]
    tdiff = row2['Zeitstr'] - row1['Zeitstr']
    if tdiff > pd.Timedelta('1 hour'):
        miss = pd.date_range(start=row1['Zeitstr'] + pd.Timedelta('1 hour'),
                                      end=row2['Zeitstr'] - pd.Timedelta('1 hour'),
                                      freq='h')
        for time in miss:
            rdf.append([time, np.nan, np.nan, np.nan])
    rdf.append([row2['Zeitstr'], row2['Passanten'], row2['Passanten ltr'], row2['Passanten rtl']])
df3 = pd.DataFrame(rdf, columns=['Zeitstr', 'Passanten', 'Passanten ltr', 'Passanten rtl'])
dfh = df3[['Zeitstr', 'Passanten','Passanten ltr','Passanten rtl']]
dfh = dfh.rename(columns={'Zeitstr': 'date'})
dfh = dfh.rename(columns={'Passanten': 'ftraffic'})
dfh = dfh.rename(columns={'Passanten ltr': 'ltr'})
dfh = dfh.rename(columns={'Passanten rtl': 'rtl'})
dfh['dow'] = dfh['date'].dt.day_name()
dfh['month'] = dfh['date'].dt.month_name()
display(dfh.head())

,date,ftraffic,ltr,rtl,dow,month
0,2024-01-01 00:00:00,1346.0,NaN,NaN,Monday,January
1,2024-01-01 01:00:00,772.0,NaN,NaN,Monday,January
2,2024-01-01 02:00:00,211.0,NaN,NaN,Monday,January
3,2024-01-01 03:00:00,173.0,NaN,NaN,Monday,January
4,2024-01-01 04:00:00,174.0,NaN,NaN,Monday,January


In [4]:
# read start and end dates of school hoildays
start=[]
end=[]
url = "https://ferien-api.de/api/v1/holidays/BY/2019"
response = requests.get(url)
ferien = response.json()
anz=len(ferien)
start.append(ferien[anz-1]['start'])
end.append(ferien[anz-1]['end'])
for iy in (2020,2021,2022,2023,2024):
    url = "https://ferien-api.de/api/v1/holidays/BY/"+str(iy)
    response = requests.get(url)
    ferien = response.json()
    anz=len(ferien)
    for ia in range(anz):
        start.append(ferien[ia]['start'])
        end.append(ferien[ia]['end'])
del start[1] # delete invalid dates
del end[1]
print(start)
print(end)

['2019-12-23', '2020-02-24', '2020-04-06', '2020-06-02', '2020-07-27', '2020-10-31', '2020-11-18', '2020-12-23', '2021-03-29', '2021-05-25', '2021-07-30', '2021-11-02', '2021-11-17', '2021-12-24', '2022-02-28', '2022-04-11', '2022-06-07', '2022-08-01', '2022-10-31', '2022-11-16', '2022-12-24', '2023-02-20', '2023-04-03', '2023-05-30', '2023-07-31', '2023-10-30', '2023-11-22', '2023-12-23', '2024-02-12', '2024-03-25', '2024-05-21', '2024-07-29', '2024-10-28', '2024-11-20', '2024-12-23']
['2020-01-04', '2020-02-28', '2020-04-18', '2020-06-13', '2020-09-07', '2020-11-06', '2020-11-18', '2021-01-09', '2021-04-10', '2021-06-04', '2021-09-13', '2021-11-05', '2021-11-17', '2022-01-08', '2022-03-04', '2022-04-23', '2022-06-18', '2022-09-12', '2022-11-04', '2022-11-16', '2023-01-07', '2023-02-24', '2023-04-15', '2023-06-09', '2023-09-11', '2023-11-03', '2023-11-22', '2024-01-05', '2024-02-16', '2024-04-06', '2024-06-01', '2024-09-09', '2024-10-31', '2024-11-20', '2025-01-03']


In [5]:
# add columns for school holidays in dataframes
sdates = pd.to_datetime(start)
edates = pd.to_datetime(end) + pd.Timedelta(hours=23, minutes=59, seconds=59)
def in_ferien(date):
    for start, end in zip(sdates, edates):
        if start <= date <= end:
            return 1
    return 0
df['s_holid'] = df['date'].apply(pd.to_datetime).apply(in_ferien)
dfh['s_holid'] = dfh['date'].apply(pd.to_datetime).apply(in_ferien)
display(df.head())
display(dfh.head())

,date,ftraffic,ltr,rtl,dow,month,s_holid
0,2020-01-01,7285.0,NaN,NaN,Wednesday,January,1
1,2020-01-02,40406.0,NaN,NaN,Thursday,January,1
2,2020-01-03,41095.0,NaN,NaN,Friday,January,1
3,2020-01-04,42096.0,NaN,NaN,Saturday,January,1
4,2020-01-05,7847.0,NaN,NaN,Sunday,January,0


,date,ftraffic,ltr,rtl,dow,month,s_holid
0,2024-01-01 00:00:00,1346.0,NaN,NaN,Monday,January,1
1,2024-01-01 01:00:00,772.0,NaN,NaN,Monday,January,1
2,2024-01-01 02:00:00,211.0,NaN,NaN,Monday,January,1
3,2024-01-01 03:00:00,173.0,NaN,NaN,Monday,January,1
4,2024-01-01 04:00:00,174.0,NaN,NaN,Monday,January,1


In [6]:
# read dates of public holidays
ftag=[]
for iy in (2020,2021,2022,2023,2024):
    url="https://date.nager.at/api/v3/PublicHolidays/"+str(iy)+"/DE"
    response = requests.get(url)
    ff = response.json()
    anz=len(ff)
    for ia in range(anz):
        counties = ff[ia]['counties']
        mm = ff[ia]['localName']
        dd = ff[ia]['date']
        if counties is None or 'DE-BY' in counties or mm == 'Mariä Himmelfahrt':
             ftag.append(dd)
print(ftag)

['2020-01-01', '2020-01-06', '2020-04-10', '2020-04-13', '2020-05-01', '2020-05-21', '2020-06-01', '2020-06-11', '2020-08-15', '2020-10-03', '2020-11-01', '2020-12-25', '2020-12-26', '2021-01-01', '2021-01-06', '2021-04-02', '2021-04-05', '2021-05-01', '2021-05-13', '2021-05-24', '2021-06-03', '2021-08-15', '2021-10-03', '2021-11-01', '2021-12-25', '2021-12-26', '2022-01-01', '2022-01-06', '2022-04-15', '2022-04-18', '2022-05-01', '2022-05-26', '2022-06-06', '2022-06-16', '2022-08-15', '2022-10-03', '2022-11-01', '2022-12-25', '2022-12-26', '2023-01-01', '2023-01-06', '2023-04-07', '2023-04-10', '2023-05-01', '2023-05-18', '2023-05-29', '2023-06-08', '2023-08-15', '2023-10-03', '2023-11-01', '2023-12-25', '2023-12-26', '2024-01-01', '2024-01-06', '2024-03-29', '2024-04-01', '2024-05-01', '2024-05-09', '2024-05-20', '2024-05-30', '2024-08-15', '2024-10-03', '2024-11-01', '2024-12-25', '2024-12-26']


In [7]:
# add columns for public holidays in dataframes
fetag = set(pd.to_datetime(ftag).date)
df['p_holid'] = df['date'].apply(lambda x: 1 if x.date() in fetag else 0)
dfh['p_holid'] = dfh['date'].apply(lambda x: 1 if x.date() in fetag else 0)
display(df.head())
display(dfh.head())

,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid
0,2020-01-01,7285.0,NaN,NaN,Wednesday,January,1,1
1,2020-01-02,40406.0,NaN,NaN,Thursday,January,1,0
2,2020-01-03,41095.0,NaN,NaN,Friday,January,1,0
3,2020-01-04,42096.0,NaN,NaN,Saturday,January,1,0
4,2020-01-05,7847.0,NaN,NaN,Sunday,January,0,0


,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid
0,2024-01-01 00:00:00,1346.0,NaN,NaN,Monday,January,1,1
1,2024-01-01 01:00:00,772.0,NaN,NaN,Monday,January,1,1
2,2024-01-01 02:00:00,211.0,NaN,NaN,Monday,January,1,1
3,2024-01-01 03:00:00,173.0,NaN,NaN,Monday,January,1,1
4,2024-01-01 04:00:00,174.0,NaN,NaN,Monday,January,1,1


In [8]:
# add column for bridge days in daily foot traffic dataframe
df['br_mo'] = 0
df['br_fr'] = 0
for i in range(len(df) - 1):
    if df.loc[i, 'dow'] == 'Monday' and df.loc[i + 1, 'p_holid'] == 1 and df.loc[i, 'p_holid'] != 1:
        df.loc[i, 'br_mo'] = 1
for i in range(1,len(df)):
    if df.loc[i, 'dow'] == 'Friday' and df.loc[i -1, 'p_holid'] == 1 and df.loc[i, 'p_holid'] != 1:
        df.loc[i, 'br_fr'] = 1
display(df.head())

,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid,br_mo,br_fr
0,2020-01-01,7285.0,NaN,NaN,Wednesday,January,1,1,0,0
1,2020-01-02,40406.0,NaN,NaN,Thursday,January,1,0,0,0
2,2020-01-03,41095.0,NaN,NaN,Friday,January,1,0,0,0
3,2020-01-04,42096.0,NaN,NaN,Saturday,January,1,0,0,0
4,2020-01-05,7847.0,NaN,NaN,Sunday,January,0,0,0,0


In [9]:
# add column for bridge days in hourly foot traffic dataframe
dfh['br_mo'] = 0
dfh['br_fr'] = 0
for i in range(len(dfh)):
    if dfh.loc[i, 'dow'] == 'Thursday' and dfh.iloc[i]['date'].hour == 23 and dfh.loc[i, 'p_holid'] == 1:
        for it in range(24):
            if dfh.loc[i+it+1, 'p_holid'] != 1:
                dfh.loc[i+it+1, 'br_fr']=1
    if dfh.loc[i, 'dow'] == 'Tuesday' and dfh.iloc[i]['date'].hour == 0 and dfh.loc[i, 'p_holid'] == 1:
        for it in range(24):
            if dfh.loc[i-(it+1), 'p_holid'] != 1:
                dfh.loc[i-(it+1), 'br_mo']=1
display(dfh.head())

,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid,br_mo,br_fr
0,2024-01-01 00:00:00,1346.0,NaN,NaN,Monday,January,1,1,0,0
1,2024-01-01 01:00:00,772.0,NaN,NaN,Monday,January,1,1,0,0
2,2024-01-01 02:00:00,211.0,NaN,NaN,Monday,January,1,1,0,0
3,2024-01-01 03:00:00,173.0,NaN,NaN,Monday,January,1,1,0,0
4,2024-01-01 04:00:00,174.0,NaN,NaN,Monday,January,1,1,0,0


In [10]:
# add columns for black week in dataframes
start=[]
end=[]
start.append('2020-11-23')
end.append('2020-11-28')
start.append('2021-11-22')
end.append('2021-11-27')
start.append('2022-11-21')
end.append('2022-11-26')
start.append('2023-11-20')
end.append('2023-11-25')
start.append('2024-11-25')
end.append('2024-11-30')
sdates = pd.to_datetime(start)
edates = pd.to_datetime(end) + pd.Timedelta(hours=23, minutes=59, seconds=59)
def in_bw(date):
    for start, end in zip(sdates, edates):
        if start <= date <= end:
            return 1
    return 0
df['bweek'] = df['date'].apply(pd.to_datetime).apply(in_bw)
dfh['bweek'] = dfh['date'].apply(pd.to_datetime).apply(in_bw)
display(df.head())
display(dfh.head())

,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid,br_mo,br_fr,bweek
0,2020-01-01,7285.0,NaN,NaN,Wednesday,January,1,1,0,0,0
1,2020-01-02,40406.0,NaN,NaN,Thursday,January,1,0,0,0,0
2,2020-01-03,41095.0,NaN,NaN,Friday,January,1,0,0,0,0
3,2020-01-04,42096.0,NaN,NaN,Saturday,January,1,0,0,0,0
4,2020-01-05,7847.0,NaN,NaN,Sunday,January,0,0,0,0,0


,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid,br_mo,br_fr,bweek
0,2024-01-01 00:00:00,1346.0,NaN,NaN,Monday,January,1,1,0,0,0
1,2024-01-01 01:00:00,772.0,NaN,NaN,Monday,January,1,1,0,0,0
2,2024-01-01 02:00:00,211.0,NaN,NaN,Monday,January,1,1,0,0,0
3,2024-01-01 03:00:00,173.0,NaN,NaN,Monday,January,1,1,0,0,0
4,2024-01-01 04:00:00,174.0,NaN,NaN,Monday,January,1,1,0,0,0


In [11]:
# add columns for festivals in dataframes
start=[]
end=[]
start.append('2020-02-23')
end.append('2020-02-23')
start.append('2021-09-10')
end.append('2021-09-12')
start.append('2021-10-30')
end.append('2021-10-31')
start.append('2022-09-09')
end.append('2022-09-11')
start.append('2022-10-29')
end.append('2022-10-30')
start.append('2023-02-19')
end.append('2023-02-19')
start.append('2023-09-08')
end.append('2023-09-10')
start.append('2023-10-28')
end.append('2023-10-29')
start.append('2024-02-11')
end.append('2024-02-11')
start.append('2024-09-06')
end.append('2024-09-08')
start.append('2024-10-26')
end.append('2024-10-27')
sdates = pd.to_datetime(start)
edates = pd.to_datetime(end) + pd.Timedelta(hours=23, minutes=59, seconds=59)
def in_fest(date):
    for start, end in zip(sdates, edates):
        if start <= date <= end:
            return 1
    return 0
df['fest'] = df['date'].apply(pd.to_datetime).apply(in_fest)
dfh['fest'] = dfh['date'].apply(pd.to_datetime).apply(in_fest)
display(df.head())
display(dfh.head())

,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid,br_mo,br_fr,bweek,fest
0,2020-01-01,7285.0,NaN,NaN,Wednesday,January,1,1,0,0,0,0
1,2020-01-02,40406.0,NaN,NaN,Thursday,January,1,0,0,0,0,0
2,2020-01-03,41095.0,NaN,NaN,Friday,January,1,0,0,0,0,0
3,2020-01-04,42096.0,NaN,NaN,Saturday,January,1,0,0,0,0,0
4,2020-01-05,7847.0,NaN,NaN,Sunday,January,0,0,0,0,0,0


,date,ftraffic,ltr,rtl,dow,month,s_holid,p_holid,br_mo,br_fr,bweek,fest
0,2024-01-01 00:00:00,1346.0,NaN,NaN,Monday,January,1,1,0,0,0,0
1,2024-01-01 01:00:00,772.0,NaN,NaN,Monday,January,1,1,0,0,0,0
2,2024-01-01 02:00:00,211.0,NaN,NaN,Monday,January,1,1,0,0,0,0
3,2024-01-01 03:00:00,173.0,NaN,NaN,Monday,January,1,1,0,0,0,0
4,2024-01-01 04:00:00,174.0,NaN,NaN,Monday,January,1,1,0,0,0,0


ftraffic = total foot traffic; ltr = pedestrians crossing from left to right; rtl = pedestrians crossing from right to left;
dow = day of week; month = month; s_holid = summer holidays (1=yes, 0=no); p_holid = public holidays (1=yes, 0=no);
br_mo = bridge days Monday (1=yes, 0=no); br_fr = brigde days Friday (1=yes, 0=no); bweek = black week (1=yes, 0=no);
fest = city festivals (1=yes, 0=no)

In [12]:
# save dataframes as .csv
df.to_csv('DATA/foottraffic_daily_schoenbornstr_prep.csv', index=False)
dfh.to_csv('DATA/foottraffic_hourly_schoenbornstr_prep.csv', index=False)